In [1]:
from qiskit import ClassicalRegister, transpile, QuantumCircuit
from qiskit.circuit.library import PhaseEstimation, QFT
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.circuit.library import UnitaryGate
import numpy as np

In [2]:
def phase_estimation(num_eval_qubits, unitary):
    """
    Phase estimation of the quantum state.
    :param total_qubits: 
    :param num_eval_qubits: 
    :param unitary: 
    :return: 
    """
    
    backend = FakeManilaV2()
    
    cl_register = ClassicalRegister(num_eval_qubits, "cl1")
    unitary_circuit = UnitaryGate(unitary)
    pe_circuit = PhaseEstimation(num_eval_qubits, unitary_circuit, iqft=None, name='QPE')
    pe_circuit.add_register(cl_register)
    transpiled_circuit = transpile(pe_circuit, backend)
    transpiled_circuit.measure(range(num_eval_qubits), cl_register)
    
    sampler = SamplerV2(backend)
    
    job = sampler.run([transpiled_circuit])

    result = job.result()[0]
    
    return result

def manual_phase_estimation(num_eval_qubits, unitary, state_qubits, state_prep):
    """
    Phase estimation with manually constructed circuit
    Args:
        num_eval_qubits: Number of evaluation qubits with initial state 0
        unitary: The unitary operator
        state_qubits: The number of qubits for preparing the state
        state_prep: The preparation circuit of the state

    Returns: DataBin object of the result

    """
    backend = FakeManilaV2()
    unitary_circuit = UnitaryGate(unitary)
    controlled_u = unitary_circuit.control(1)
    t = num_eval_qubits
    qpe = QuantumCircuit(t + state_qubits, num_eval_qubits)
    qpe.append(state_prep, [t + i for i in range(state_qubits)])
    for qubit in range(t):
        qpe.h(qubit)

    repetitions = 1
    for counting_qubit in range(t):
        for i in range(repetitions):
            qpe.append(controlled_u, [counting_qubit, t])
            # qpe.cp(math.pi/8, counting_qubit, t)
        repetitions *= 2

    myQft = QFT(num_qubits=t, do_swaps=True, inverse=True, insert_barriers=True,
                name="myQFT")
    qpe.append(myQft, range(num_eval_qubits))

    for n in range(num_eval_qubits):
        qpe.measure(n, n)

    qpe.draw()

    transpiled_circuit = transpile(qpe, backend)
    sampler = SamplerV2(backend)

    job = sampler.run([transpiled_circuit])

    result = job.result()[0]
    print(qpe)

    return result

In [3]:
num_eval_qubits = 2
cl_register = ClassicalRegister(num_eval_qubits, "cl1")
unitary = np.array([[1, 0], [0, np.exp(np.pi * 1j / 8)]])
unitary_circuit = UnitaryGate(unitary)
pe_circuit = PhaseEstimation(num_eval_qubits, unitary_circuit, iqft=None, name='QPE')
pe_circuit.add_register(cl_register)
pe_circuit.measure(range(num_eval_qubits), cl_register)

pe_circuit.draw("mpl")

print(pe_circuit)

        ┌──────┐┌─┐   
eval_0: ┤0     ├┤M├───
        │      │└╥┘┌─┐
eval_1: ┤1 QPE ├─╫─┤M├
        │      │ ║ └╥┘
     q: ┤2     ├─╫──╫─
        └──────┘ ║  ║ 
 cl1: 2/═════════╩══╩═
                 0  1 


In [4]:
pub_result = phase_estimation(num_eval_qubits, unitary)

C:\Users\jhlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\qiskit_ibm_runtime\fake_provider\fake_backend.py:366: RuntimeWarning: Aer not found, using qiskit.BasicSimulator and no noise.
  warnings.warn(


In [5]:
pub_result.data.cl1.array

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [6]:
state_qubits = 1
state_prep = QuantumCircuit(state_qubits)
state_prep.x(0)
manual_phase_estimation(3, unitary, state_qubits, state_prep).data.c.array

          ┌───┐                                                            »
q_0: ─────┤ H ├──────────■─────────────────────────────────────────────────»
          ├───┤          │                                                 »
q_1: ─────┤ H ├──────────┼──────────■──────────■───────────────────────────»
          ├───┤          │          │          │                           »
q_2: ─────┤ H ├──────────┼──────────┼──────────┼──────────■──────────■─────»
     ┌────┴───┴────┐┌────┴────┐┌────┴────┐┌────┴────┐┌────┴────┐┌────┴────┐»
q_3: ┤ circuit-255 ├┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├┤ Unitary ├»
     └─────────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
c: 3/══════════════════════════════════════════════════════════════════════»
                                                                           »
«                           ┌────────┐┌─┐      
«q_0: ──────────────────────┤0       ├┤M├──────
«                           │        │└╥┘┌─┐   
«q_1: ───

array([[0],
       [0],
       [2],
       ...,
       [1],
       [1],
       [2]], dtype=uint8)